In [14]:
!pip3 install netCDF4 matplotlib
!pip3 install pandas
!pip3 install matplotlib

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [17]:
import os
import netCDF4
import gzip
import pandas as pd
import datetime

# Folder containing the files
folder_files = './all_CDFs/6'

dataframes = []

for file in os.listdir(folder_files):
    if file.endswith('.nc.gz'):
        complete_file = os.path.join(folder_files, file)
        with gzip.open(complete_file, 'rb') as gz:
            with netCDF4.Dataset('dummy', mode='r', memory=gz.read()) as nc:
                time = nc.variables['time'][:]
                bx_gsm = nc.variables['bx_gsm'][:]
                by_gsm = nc.variables['by_gsm'][:]
                bz_gsm = nc.variables['bz_gsm'][:]
                bt = nc.variables['bt'][:]

        df = pd.DataFrame({'Timesteps': time, 'bx_gsm': bx_gsm, 'by_gsm': by_gsm, 'bz_gsm': bz_gsm, 'bt': bt})
        df['Timesteps'] = [datetime.datetime.fromtimestamp(ts / 1000) for ts in df['Timesteps']]
        df['Timesteps'] = pd.to_datetime(df['Timesteps'])
        df.set_index('Timesteps', inplace=True)
        dataframes.append(df)

concatenated_df = pd.concat(dataframes)
resampled_df = concatenated_df.resample('3H').mean()
resampled_df.to_csv('./results_csv/partials/formatted_combined_data_6.csv', index=True)
